<a href="https://colab.research.google.com/github/Metusalen/agora-vai/blob/main/Metusa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas numpy pycaret scikit-learn



In [13]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.4 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
from pycaret.classification import *
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

# 1. Carregamento de Dados
# Carregar dados de 2012 a 2016
data_2012_2016 = pd.concat([pd.read_csv(f'{ano} QSE1.csv', sep=';') for ano in range(2012, 2017)])

# Carregar dados de 2017
data_2017 = pd.read_csv('/content/2017 QSE1.csv', sep=';')  # Substitua pelo seu caminho
print("Colunas do DataFrame 2012-2016:", data_2012_2016.columns.tolist())
print("Colunas do DataFrame 2017:", data_2017.columns.tolist())

# 2. Normalização dos Nomes das Colunas
data_2012_2016.columns = data_2012_2016.columns.str.strip()
data_2017.columns = data_2017.columns.str.strip()

# 3. Pré-processamento dos Dados
# Verificar dados faltantes
print("Dados faltantes no conjunto de treino:\n", data_2012_2016.isnull().sum())
print("Dados faltantes no conjunto de teste:\n", data_2017.isnull().sum())

# Remover ou tratar dados faltantes se necessário
data_2012_2016.dropna(inplace=True)
data_2017.dropna(inplace=True)


# 4. Separação de Recursos e Alvo
target_column = 'MORTE'  # Nome da coluna alvo
if target_column in data_2012_2016.columns and target_column in data_2017.columns:
    X_train = data_2012_2016.drop(target_column, axis=1)
    y_train = data_2012_2016[target_column]
    X_test = data_2017.drop(target_column, axis=1)
    y_test = data_2017[target_column]
else:
    raise ValueError(f"A coluna '{target_column}' não está presente em ambos os DataFrames.")

# 5. Redefinir o índice
data_2012_2016.reset_index(drop=True, inplace=True)
data_2017.reset_index(drop=True, inplace=True)

# 6. Inicialização do PyCaret
clf = setup(data=data_2012_2016, target=target_column, session_id=123, html=False)

# 7. Comparação de Modelos
# Remover 'nn' da lista de modelos
best_model = compare_models(include=['lr', 'dt', 'rf', 'xgboost', 'lightgbm',], n_select=4)

# 8. Ajuste de Hiperparâmetros
tuned_models = [tune_model(model, n_iter=50, optimize='AUC') for model in best_model]

# 9. Avaliação do Desempenho
for model in tuned_models:
    evaluate_model(model)

    # Predição no conjunto de teste
    y_pred = predict_model(model, data=X_test)

    # Calcular métricas
    auc = roc_auc_score(y_test, y_pred['Label'])
    f1 = f1_score(y_test, y_pred['Label'])
    accuracy = accuracy_score(y_test, y_pred['Label'])

    print(f'Modelo: {model}, AUC: {auc:.4f}, F1: {f1:.4f}, Precisão: {accuracy:.4f}')


Colunas do DataFrame 2012-2016: ['ESFERA', 'LOCNASC', 'IDADEMAE', 'SEXO_DN', 'APGAR1', 'APGAR5', 'PESO', 'IDANOMAL', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'GESTACAO', 'ESCMAE2010', 'RACACORMAE', 'CONSPRENAT', 'QTDFILVIVO', 'QTDFILMORT', 'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'TPNASCASSI', 'MORTE']
Colunas do DataFrame 2017: ['ESFERA', 'LOCNASC', 'IDADEMAE', 'SEXO_DN', 'APGAR1', 'APGAR5', 'PESO', 'IDANOMAL', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'GESTACAO', 'ESCMAE2010', 'RACACORMAE', 'CONSPRENAT', 'QTDFILVIVO', 'QTDFILMORT', 'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'TPNASCASSI', 'MORTE']
Dados faltantes no conjunto de treino:
 ESFERA         6599
LOCNASC           0
IDADEMAE          0
SEXO_DN           0
APGAR1         2112
APGAR5         2024
PESO             27
IDANOMAL        251
GRAVIDEZ         58
PARTO            64
CONSULTAS         0
GESTACAO       1892
ESCMAE2010     3000
RACACORMAE     3063
CONSPRE


Processing: 100%|██████████| 28/28 [05:06<00:00,  8.84s/it]
                                                           

                                    Model  Accuracy     AUC  Recall   Prec.  \
rf               Random Forest Classifier    0.9910  0.8670  0.2430  0.7701   
xgboost         Extreme Gradient Boosting    0.9903  0.8830  0.2686  0.6342   
lr                    Logistic Regression    0.9902  0.8910  0.1890  0.6799   
lightgbm  Light Gradient Boosting Machine    0.9900  0.8969  0.2955  0.5778   
dt               Decision Tree Classifier    0.9824  0.6574  0.3253  0.2587   

              F1   Kappa     MCC  TT (Sec)  
rf        0.3671  0.3638  0.4277     9.464  
xgboost   0.3763  0.3722  0.4081     1.266  
lr        0.2930  0.2896  0.3528     7.827  
lightgbm  0.3895  0.3850  0.4079     8.211  
dt        0.2874  0.2786  0.2809     0.632  



Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 50 candidates, totalling 500 fits


KeyboardInterrupt: 